### Imports

In [3]:
import pandas as pd

# Onze custom hupmlsetupexplained package
import libs.hupmlsetupexplained as hupmlsetupexplained

### Hupmlsetupexplained machine learning dataframes (`MlDataFrame`)

Hieronder zie je hoe we met de reguliere pandas functie de data inladen. Dit is het data object, wat een regulier pandas `DataFrame` is. In de tweede regel code converteren we het pandas dataframe naar een hupml machine learning dataframe.

In [4]:
data = pd.read_csv('datasets/crime.csv', engine='python')
df = hupmlsetupexplained.MlDataFrame(data=data)

We kunnen er zelfs voor kiezen om een hupmlsetupexplained.read_csv te maken die een csv laadt naar een MlDataFrame, je krijgt dan zoiets als:

In [ ]:
data = hupmlsetupexplained.read_csv('datasets/crime.csv')

Nadeel hiervan is dat je iedere pandas utility functie die je ook voor je eigen dataframe wil gebruiken, om moet schrijven naar iets van je zelf.

De `MlDataFrame` class erft van pandas' `DataFrame`, dus `MlDataFrame` _is_ een `DataFrame` (definitie van overerving). Dit betekent in de praktijk dat we alle functies kunnen gebruiken van pandas, **plus** onze eigen functies. Hoe ziet er dan aan de achterkant uit? Hieronder zie je de MlDataFrame class:

In [ ]:
# Custom pandas DataFrame
class MlDataFrame(DataFrame):
    # Abstract method of the Pandas DataFrame class: just calls super class
    @property
    def _constructor_expanddim(self):
        return super()._constructor_expanddim(self)

    # A lot of methods in the DataFrame class return a DataFrame using the _constructor method
    # Every time a new dataframe is created, we return the inherited dataframe
    @property
    @abstractmethod
    def _constructor(self):
        return MlDataFrame

    # When slicing methods are called, return custom (inherited) Series object
    @property
    @abstractmethod
    def _constructor_sliced(self):
        return MlSeries

    @property
    def get_df(self):
        return DataFrame(self)
    
    # Custom method
    def return_first_column(self):
        return self.iloc[:, 0]

# Custom pandas Series
class MlSeries(Series):
    @property
    def _constructor(self):
        # return MlSeries
        # For now the same as Pandas Series
        # If custom methods are needed, uncomment first line
        return Series

Een voorbeeld hoe dit te gebruiken is in de praktijk:

In [ ]:
data = pd.read_csv('datasets/crime.csv', engine='python')
df = hupmlsetupexplained.MlDataFrame(data=data)

# Call custom method
df.return_first_column()

Zoals je ziet is deze syntax zeer makkelijk te gebruiken en werkt alsof je een normale pandas dataframe gebruikt. Als je dus een vers-van-de-pers-consultant bent, is dit zeer makkelijk te gebruiken.

### Nog een stapje verder, overerving van `MlDataFrame`

Als we nu nog een stapje verder kijken, kunnen we nog meer soorten `DataFrames` toevoegen aan onze `hupmlsetupexplained` package. Bijvoorbeeld een `DataFrame` die specifiek dealt met timeseries. Dit `DataFrame` heet `TimeDataFrame` en erft van `MlDataFrame` wat op haar beurt weer erft van `DataFrame`, dus ook `TimeDataFrame` is uiteindelijk een pandas `DataFrame`. 

Inladen gaat dan weer precies hetzelfde, alleen kunnen we nu een functie gebruiken die alleen specifiek voor timeseries aanwezig is. Zo gaan we er bij een timeseries dataframe vanuit dat er één of meerdere datetime kolommen zijn. Dit kunnen we nu direct aangeven met een simpele functie:

In [2]:
data = pd.read_csv('datasets/crime.csv', engine='python')
df = hupmlsetupexplained.TimeDataFrame(data=data)

# Call custom method
df.convert_datetime_cols(['OCCURRED_ON_DATE', 'date_col1', 'date_col2', 'date_col3', 'date_col4', 'date_col5'])
df

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,...,UCR_PART,STREET,Lat,Long,Location,date_col1,date_col2,date_col3,date_col4,date_col5
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,...,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)",2011-01-01,2011-01-01,2011-01-01,2011-01-01,2011-01-01
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,...,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)",2011-01-02,2011-01-02,2011-01-02,2011-01-02,2011-01-02
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,...,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)",2011-01-03,2011-01-03,2011-01-03,2011-01-03,2011-01-03
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,...,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)",2011-01-04,2011-01-04,2011-01-04,2011-01-04,2011-01-04
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,...,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)",2011-01-05,2011-01-05,2011-01-05,2011-01-05,2011-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,I182069846,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,A1,111,NaN,2018-08-31 02:15:00,2018,8,...,Part One,WEST ST,42.354591,-71.062190,"(42.35459106, -71.06219010)",2013-09-22,2013-09-22,2013-09-22,2013-09-22,2013-09-22
996,I182069844,3115,Investigate Person,INVESTIGATE PERSON,E13,577,NaN,2018-08-31 01:30:00,2018,8,...,Part Three,ARCADIA ST,42.316172,-71.100483,"(42.31617241, -71.10048350)",2013-09-23,2013-09-23,2013-09-23,2013-09-23,2013-09-23
997,I182069842,3410,Towed,TOWED MOTOR VEHICLE,A1,118,NaN,2018-08-31 02:14:00,2018,8,...,Part Three,BOYLSTON ST,42.352418,-71.065255,"(42.35241815, -71.06525499)",2013-09-24,2013-09-24,2013-09-24,2013-09-24,2013-09-24
998,I182069842,2906,Violations,VAL - OPERATING UNREG/UNINS Ã‚Â CAR,A1,118,NaN,2018-08-31 02:14:00,2018,8,...,Part Two,BOYLSTON ST,42.352418,-71.065255,"(42.35241815, -71.06525499)",2013-09-25,2013-09-25,2013-09-25,2013-09-25,2013-09-25


Super nice right? Echter zitten er een paar haken en ogen aan deze manier van overerving van het pandas `DataFrame`. De documentatie van pandas [benoemt dit ook](https://pandas.pydata.org/pandas-docs/stable/development/extending.html), echter niet in veel detail _wat_ voor problemen kunnen ontstaan. Daar ben ik echter inmiddels tegen aan gelopen.

### Problemen met direct overerving van het pandas `DataFrame`

Stel nu dat we direct de datetime kolommen conversie hadden willen doen bij het aanmaken van het object. Logischerwijs zou dat er dan uitzien als:
```
df = hupmlsetupexplained.TimeDataFrame(data=data, datetime_cols=['OCCURRED_ON_DATE', 'date_col1', 'date_col2', 
                                                   'date_col3', 'date_col4', 'date_col5'])
```
Oftewel, we geven direct de datetime kolommen mee, want het is toch een timeseries. Nu ontstaan echter de problemen. In de package zit ook een class `IssuesWithInitTimeDataFrame`. Die ziet er zo uit:

In [ ]:
class IssuesWithInitTimeDataFrame(MlDataFrame):
    def __init__(self, datetime_cols, data=None, index=None, columns=None, dtype=None, copy=False):
        super().__init__(data=data, index=index, columns=columns, dtype=dtype, copy=copy)
        self.convert_datetime_cols(datetime_cols=datetime_cols)

    @property
    def _constructor(self):
        return IssuesWithInitTimeDataFrame

    @property
    def _constructor_sliced(self):
        return super()._constructor_sliced

    def convert_datetime_cols(self, datetime_cols):
        # Convert all date/datetime cols to datetimes
        if not isinstance(datetime_cols, list):
            self[datetime_cols] = pd.to_datetime(self[datetime_cols])
        else:
            for col in datetime_cols:
                if not isinstance(self[col], pd.datetime):
                    self[col] = pd.to_datetime(self[col])

Precies hetzelfde, op de `__init__` functie na. Nu even kijken wat er gebeurt als we het dit `DataFrame` gewoon willen printen:

In [4]:
data = pd.read_csv('datasets/crime.csv', engine='python')
df = hupmlsetupexplained.IssuesWithInitTimeDataFrame(data=data, datetime_cols=['OCCURRED_ON_DATE', 'date_col1', 'date_col2', 
                                                   'date_col3', 'date_col4', 'date_col5'])
print(df)

KeyError: BlockManager
Items: Index(['INCIDENT_NUMBER', 'OFFENSE_CODE', 'OFFENSE_CODE_GROUP',
       'OFFENSE_DESCRIPTION', 'DISTRICT', 'REPORTING_AREA', 'SHOOTING',
       'OCCURRED_ON_DATE', 'YEAR', 'MONTH'],
      dtype='object')
Axis 1: RangeIndex(start=0, stop=1000, step=1)
ObjectBlock: [0, 2, 3, 4, 5], 5 x 1000, dtype: object
IntBlock: [1, 8, 9], 3 x 1000, dtype: int64
FloatBlock: slice(6, 7, 1), 1 x 1000, dtype: float64
DatetimeBlock: slice(7, 8, 1), 1 x 1000, dtype: datetime64[ns]

Allemaal gekke errors. Onderwater doet pandas toch nog dingen die we niet verwachten. Wat dit precies allemaal is, is lastig te zeggen, maar het zit duidelijk in de `__init__` functie. Het lijkt er op dat onderwater de `__init__` functie wordt aangeroepen (voor de echte techneuten, dit gaat via de functie `__repr__` zoals bij ieder object in Python) en er positionele argumenten worden meegegeven. Aangezien `datetime_cols` op positie 1 staat, krijgt deze op de een of andere manier de `BlockManager` mee. We kunnen dit probleem omzeilen door datetime_cols als keyword argument achteraan te zetten en in de `__init__` functie te checken op `None` type:

In [ ]:
class SolvedIssuesWithInitTimeDataFrame(MlDataFrame):
    def __init__(self, data=None, index=None, columns=None, dtype=None, copy=False, datetime_cols=None):
        super().__init__(data=data, index=index, columns=columns, dtype=dtype, copy=copy)
        self.datetime_cols = datetime_cols

        if datetime_cols is not None:
            self.convert_datetime_cols()

    # A lot of methods in the DataFrame class return a DataFrame using the _constructor method
    # Every time a new dataframe is created, we return the inherited dataframe
    @property
    def _constructor(self):
        return SolvedIssuesWithInitTimeDataFrame

    # When slicing methods are called, return custom (inherited) Series object
    @property
    def _constructor_sliced(self):
        return super()._constructor_sliced

    def convert_datetime_cols(self):
        # Convert all date/datetime cols to datetimes
        if not isinstance(self.datetime_cols, list):
            self[self.datetime_cols] = pd.to_datetime(self[self.datetime_cols])
        else:
            for col in self.datetime_cols:
                if not isinstance(self[col], pd.datetime):
                    self[col] = pd.to_datetime(self[col])

We kunnen dan op de volgende manier aanroepen:

In [4]:
data = pd.read_csv('datasets/crime.csv', engine='python')
df = hupmlsetupexplained.SolvedIssuesWithInitTimeDataFrame(data=data, datetime_cols=['OCCURRED_ON_DATE', 'date_col1', 'date_col2', 
                                                   'date_col3', 'date_col4', 'date_col5'])
df

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,...,UCR_PART,STREET,Lat,Long,Location,date_col1,date_col2,date_col3,date_col4,date_col5
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,...,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)",2011-01-01,2011-01-01,2011-01-01,2011-01-01,2011-01-01
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,...,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)",2011-01-02,2011-01-02,2011-01-02,2011-01-02,2011-01-02
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,...,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)",2011-01-03,2011-01-03,2011-01-03,2011-01-03,2011-01-03
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,...,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)",2011-01-04,2011-01-04,2011-01-04,2011-01-04,2011-01-04
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,...,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)",2011-01-05,2011-01-05,2011-01-05,2011-01-05,2011-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,I182069846,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,A1,111,NaN,2018-08-31 02:15:00,2018,8,...,Part One,WEST ST,42.354591,-71.062190,"(42.35459106, -71.06219010)",2013-09-22,2013-09-22,2013-09-22,2013-09-22,2013-09-22
996,I182069844,3115,Investigate Person,INVESTIGATE PERSON,E13,577,NaN,2018-08-31 01:30:00,2018,8,...,Part Three,ARCADIA ST,42.316172,-71.100483,"(42.31617241, -71.10048350)",2013-09-23,2013-09-23,2013-09-23,2013-09-23,2013-09-23
997,I182069842,3410,Towed,TOWED MOTOR VEHICLE,A1,118,NaN,2018-08-31 02:14:00,2018,8,...,Part Three,BOYLSTON ST,42.352418,-71.065255,"(42.35241815, -71.06525499)",2013-09-24,2013-09-24,2013-09-24,2013-09-24,2013-09-24
998,I182069842,2906,Violations,VAL - OPERATING UNREG/UNINS Ã‚Â CAR,A1,118,NaN,2018-08-31 02:14:00,2018,8,...,Part Two,BOYLSTON ST,42.352418,-71.065255,"(42.35241815, -71.06525499)",2013-09-25,2013-09-25,2013-09-25,2013-09-25,2013-09-25


Dan nog één laatste opmerking: Met de huidige setup gaan we er vanuit dat pandas `DataFrame` nooit geupdate wordt en we dus altijd de argumenten `data=None, index=None, columns=None, dtype=None, copy=False` hebben, niets meer, niets minder. We kunnen echter alle toekomstige argumenten doorgeven door alle overige argumenten af te vangen. Als inspiratie, is het goed om eens naar [GeoPandas te kijken.]() Je ziet hoe zij de overerving van Pandas hebben gedaan en hoe ze de argumenten afvangen, zelfs future proof. We kunnen dus alle argumenten vervangen met `*args, **kwargs`. Er is hierbij één echt nadeel, in PyCharm en vergelijkbare IDE's, kan je zien wat er aan argumenten je functie in moet (bijv. Ctrl+p binnen PyCharm), dit kan uiteraard niet meer. Het resultaat is dan de volgende class:

In [ ]:
class NiceTimeDataFrame(MlDataFrame):
    def __init__(self, *args, **kwargs):
        self.datetime_cols = kwargs.pop("datetime_cols", None)
        super().__init__(*args, **kwargs)

        if self.datetime_cols is not None:
            self.convert_datetime_cols()

    @property
    def _constructor(self):
        return NiceTimeDataFrame

    @property
    def _constructor_sliced(self):
        return super()._constructor_sliced

    def convert_datetime_cols(self):
        # Convert all date/datetime cols to datetimes
        if not isinstance(self.datetime_cols, list):
            self[self.datetime_cols] = pd.to_datetime(self[self.datetime_cols])
        else:
            for col in self.datetime_cols:
                if not isinstance(self[col], pd.datetime):
                    self[col] = pd.to_datetime(self[col])

In [3]:
data = pd.read_csv('datasets/crime.csv', engine='python')
df = hupmlsetupexplained.NiceTimeDataFrame(data=data, datetime_cols=['OCCURRED_ON_DATE', 'date_col1', 'date_col2', 
                                                   'date_col3', 'date_col4', 'date_col5'])
df

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,...,UCR_PART,STREET,Lat,Long,Location,date_col1,date_col2,date_col3,date_col4,date_col5
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,...,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)",2011-01-01,2011-01-01,2011-01-01,2011-01-01,2011-01-01
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,...,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)",2011-01-02,2011-01-02,2011-01-02,2011-01-02,2011-01-02
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,...,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)",2011-01-03,2011-01-03,2011-01-03,2011-01-03,2011-01-03
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,...,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)",2011-01-04,2011-01-04,2011-01-04,2011-01-04,2011-01-04
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,...,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)",2011-01-05,2011-01-05,2011-01-05,2011-01-05,2011-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,I182069846,413,Aggravated Assault,ASSAULT - AGGRAVATED - BATTERY,A1,111,NaN,2018-08-31 02:15:00,2018,8,...,Part One,WEST ST,42.354591,-71.062190,"(42.35459106, -71.06219010)",2013-09-22,2013-09-22,2013-09-22,2013-09-22,2013-09-22
996,I182069844,3115,Investigate Person,INVESTIGATE PERSON,E13,577,NaN,2018-08-31 01:30:00,2018,8,...,Part Three,ARCADIA ST,42.316172,-71.100483,"(42.31617241, -71.10048350)",2013-09-23,2013-09-23,2013-09-23,2013-09-23,2013-09-23
997,I182069842,3410,Towed,TOWED MOTOR VEHICLE,A1,118,NaN,2018-08-31 02:14:00,2018,8,...,Part Three,BOYLSTON ST,42.352418,-71.065255,"(42.35241815, -71.06525499)",2013-09-24,2013-09-24,2013-09-24,2013-09-24,2013-09-24
998,I182069842,2906,Violations,VAL - OPERATING UNREG/UNINS Ã‚Â CAR,A1,118,NaN,2018-08-31 02:14:00,2018,8,...,Part Two,BOYLSTON ST,42.352418,-71.065255,"(42.35241815, -71.06525499)",2013-09-25,2013-09-25,2013-09-25,2013-09-25,2013-09-25


#### Conclusie
Nog even alle nadelen van deze methode op een rijtje:
* Alle util functies buiten de pandas `DataFrame` class om zullen allemaal herschreven worden als het toepasbaar moet worden op een pandas
* Bij updates van pandas moet er goed gekeken worden of `hupmlsetupexplained` blijft werken
* De `__init__` methode kan niet gebruikt worden zoals je dit zou verwachten. Er is hier omheen te werken zoals hierboven uitgelegd. Het is mogelijk dat er zich op de lange termijn hier nog meer problemen voor doen, aangezien dit niet goed gedocumenteerd is. Wellicht is het de moeite uit om onderzoek te doen naar alle effecten.

Punt 1 en 2 vind ik nog niet eens een heel groot probleem, punt 3 wel. Ik kan mij voorstellen dat hoe uitgebreider onze functies worden, hoe van groter belang dit wordt.

### Werken met decorators

[Een oplossing](https://pandas.pydata.org/pandas-docs/stable/development/extending.html) voor de drie nadelen hierboven genoemd, is om niet over te erven van pandas `DataFrame`, maar te "extenden" met custom "accessors". Je krijgt dan de volgende class structuur:

In [ ]:
@pd.api.extensions.register_dataframe_accessor("mlframe")
class MlDataFrame2:
    def __init__(self, pandas_obj):
        self._obj = pandas_obj

    # Custom method
    def return_first_column(self):
        return self._obj.iloc[:, 0]

Dit ziet er een stuk makkelijker uit, omdat we de drie verplichte functies niet meer hebben. Gebruik is dan als volgt:

In [3]:
data = pd.read_csv('datasets/crime.csv', engine='python')
data.mlframe.return_first_column()

0      I182070945
1      I182070943
2      I182070941
3      I182070940
4      I182070938
          ...    
995    I182069846
996    I182069844
997    I182069842
998    I182069842
999    I182069841
Name: INCIDENT_NUMBER, Length: 1000, dtype: object

Ondanks dat dit de drie nadelen oplost en specifiek het grootste probleem met de `__init__` functie, krijgen we weer een syntax die lastiger te begrijpen is voor vers-van-de-pers-consultant. Bovendien wordt hier de `__init__` functie alsnog pas aangeroepen, als `mlframe` op wat voor manier dan ook (dus bijvoorbeeld via `.return_first_column()`). Enige voordeel hier, is dat het de gebruiker forceert deze functie uit, echter kunnen er geen argumenten meegegeven worden aan de `__init__` functie.

### Conclusie

Na alle voor en nadelen hierboven bekeken te hebben, lijkt dat directe overerving met zorgvuldige implementatie voor nu het beste is.